In [2]:
import numpy as np

size = 8
board = np.full((size, size), "-")


def draw_board():
    print("  " + " ".join(str(i) for i in range(size)))
    for i in range(size):
        print(str(i) + " " + " ".join(board[i]))

In [3]:
def get_lines(board):
    size = len(board)
    lines = []
    coords = []

    # Xử lý các hàng
    for i in range(size):
        lines.append(board[i])
        coords.append([(i, j) for j in range(size)])  # Lưu tọa độ của mỗi ô trong hàng

    # Xử lý các cột
    for j in range(size):
        column = []
        column_coords = []
        for i in range(size):
            column.append(board[i][j])
            column_coords.append((i, j))  # Lưu tọa độ của mỗi ô trong cột
        lines.append(column)
        coords.append(column_coords)

    return lines, coords


def get_diagonals(board):
    size = len(board)
    diagonals = []
    diag_coords = []

    # Đường chéo từ góc trên bên trái đến góc dưới bên phải
    for d in range(-size + 1, size):
        diag = []
        diag_coord = []
        for i in range(max(0, d), min(size, size + d)):
            diag.append(board[i][i - d])
            diag_coord.append((i, i - d))
        if len(diag) >= 4:
            diagonals.append(diag)
            diag_coords.append(diag_coord)

    # Đường chéo từ góc trên bên phải đến góc dưới bên trái
    for d in range(3, size * 2 - 4):
        diag = []
        diag_coord = []
        for i in range(max(0, d - size + 1), min(size, d + 1)):
            diag.append(board[i][d - i])
            diag_coord.append((i, d - i))
        if len(diag) >= 4:
            diagonals.append(diag)
            diag_coords.append(diag_coord)

    return diagonals, diag_coords


def get_all_lines():
    lines, coords = get_lines(board)
    diag_lines, diag_coords = get_diagonals(board)
    lines.extend(diag_lines)
    coords.extend(diag_coords)
    return (lines, coords)

In [4]:
board[0, 0] = "x"
board[0, 1] = "x"
board[0, 2] = "x"
board[0, 3] = "-"

board[1, 0] = "o"
board[1, 1] = "o"
board[1, 2] = "o"

board[4, 4] = "x"
board[5, 5] = "x"
board[6, 6] = "x"
board[7, 7] = "x"

In [5]:
player = "x"
opp_player = "o"
index = 1
draw_board()

for line, coord in zip(*get_all_lines()):
    for i in range(len(line) - 3):
        sub_line = np.array(line[i : i + 4])
        sub_coord = coord[i : i + 4]

        count_player = np.count_nonzero(sub_line == player)
        count_opp_player = np.count_nonzero(sub_line == opp_player)
        count_empty = np.count_nonzero(sub_line == "-")

        print(sub_line, sub_coord, count_player, count_opp_player, count_empty)

  0 1 2 3 4 5 6 7
0 x x x - - - - -
1 o o o - - - - -
2 - - - - - - - -
3 - - - - - - - -
4 - - - - x - - -
5 - - - - - x - -
6 - - - - - - x -
7 - - - - - - - x
['x' 'x' 'x' '-'] [(0, 0), (0, 1), (0, 2), (0, 3)] 3 0 1
['x' 'x' '-' '-'] [(0, 1), (0, 2), (0, 3), (0, 4)] 2 0 2
['x' '-' '-' '-'] [(0, 2), (0, 3), (0, 4), (0, 5)] 1 0 3
['-' '-' '-' '-'] [(0, 3), (0, 4), (0, 5), (0, 6)] 0 0 4
['-' '-' '-' '-'] [(0, 4), (0, 5), (0, 6), (0, 7)] 0 0 4
['o' 'o' 'o' '-'] [(1, 0), (1, 1), (1, 2), (1, 3)] 0 3 1
['o' 'o' '-' '-'] [(1, 1), (1, 2), (1, 3), (1, 4)] 0 2 2
['o' '-' '-' '-'] [(1, 2), (1, 3), (1, 4), (1, 5)] 0 1 3
['-' '-' '-' '-'] [(1, 3), (1, 4), (1, 5), (1, 6)] 0 0 4
['-' '-' '-' '-'] [(1, 4), (1, 5), (1, 6), (1, 7)] 0 0 4
['-' '-' '-' '-'] [(2, 0), (2, 1), (2, 2), (2, 3)] 0 0 4
['-' '-' '-' '-'] [(2, 1), (2, 2), (2, 3), (2, 4)] 0 0 4
['-' '-' '-' '-'] [(2, 2), (2, 3), (2, 4), (2, 5)] 0 0 4
['-' '-' '-' '-'] [(2, 3), (2, 4), (2, 5), (2, 6)] 0 0 4
['-' '-' '-' '-'] [(2, 4), (2, 5), (2, 6

In [10]:
player_score = 0
opponent = 0
total_score = 0

scores = {1: 1, 2: 10, 3: 50, 4: 1000}
lines = []
size = len(board)

for i in range(size):
    lines.append(board[i, :])
    lines.append(board[:, i])

lines.extend([board.diagonal(i) for i in range(-size + 1, size)])
lines.extend([board[:, ::-1].diagonal(i) for i in range(-size + 1, size)])
lines = [line for line in lines if len(line) >= 4]

for line in lines:
    for i in range(len(line) - 3):
        sub_line = np.array(line[i : i + 4])
        segment = "".join(sub_line)
        
        print(segment.count("x"))

3
2
1
0
0
1
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
0
1
1
1
1
0
0
1
1
1
0
0
1
1
1
0
0
0
1
1
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
2
3
3
1
0
0
0
1
0
0
0
0
0
0
1
1
0
0
0
1
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [7]:
a = hash(board.tostring())
board[7, 7] = "o"
b = hash(board.tostring())

a, b

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_14840\1132460959.py:1: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  a = hash(board.tostring())
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_14840\1132460959.py:3: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  b = hash(board.tostring())


(3218696016742016157, -1627292397765113840)

In [8]:
line_score = 0
target_length = 4
line_str = "".join(line)
patterns = {player * 2: 10, player * 3: 50, player * 4: 1000}
for pattern, points in patterns.items():
    if pattern in line_str:
        line_score += points